# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Шевчук Анастасия Олеговна`

Группа: `MADE-DS-12`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

In [9]:
import numpy as np
import pandas as pd
import folium
import requests
import json 

from random import uniform
import geopandas as gpd
from datetime import datetime, timedelta
from shapely.geometry import Point, Polygon
from haversine import haversine, Unit
from folium import plugins

In [10]:
# Get data
url_prefix = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
source_url = f'{url_prefix}/world-countries.json'

response = requests.get(source_url)
all_countries_data = json.loads(response.content.decode())

In [11]:
# Extract Afghanistan data
for item in all_countries_data['features']:
    if item['properties']['name'] == 'Afghanistan':
        afghanistan_data = item
        break

In [12]:
# Function for generating a random point from the area bounded by the given lat and lon
def random_point(lat_1, lat_2, lon_1, lon_2):
    return Point(uniform(lat_1, lat_2), uniform(lon_1, lon_2))

In [13]:
def calculate_dist_to_afghan_center_in_meters(x, y):
    return haversine((x, y), afghanistan_center_coordinates, unit = Unit.METERS)

In [14]:
def calculate_distances(point):
    result = []
    result.append(calculate_dist_to_afghan_center_in_meters(point.x + abs(lat_2 - lat_1) // 4, point.y))
    result.append(calculate_dist_to_afghan_center_in_meters(point.x,  point.y - abs(lon_1 - lon_2) // 4))
    result.append(calculate_dist_to_afghan_center_in_meters(point.x, point.y + abs(lon_1 - lon_2) // 4))
    result.append(calculate_dist_to_afghan_center_in_meters(point.x - abs(lat_2 - lat_1) // 4, point.y))
    return result

In [15]:
afghanistan_polygon = Polygon(afghanistan_data['geometry']['coordinates'][0])
afghanistan_center_coordinates = [afghanistan_polygon.centroid.y, afghanistan_polygon.centroid.x]
lat_1, lat_2, lon_1, lon_2 = -90, 90, -180, 180
point = random_point(lat_1, lat_2, lon_1, lon_2)

In [16]:
points = []
while (not Point(point.y,point.x).within(afghanistan_polygon)):
    points.append(point)
    north, west, east, south = calculate_distances(point)
    if (west < east):
        lon_2 = point.y
    else:
        lon_1 = point.y
    if (north < south):
        lat_1 = point.x
    else:
        lat_2 = point.x
    point = random_point(lat_1, lat_2, lon_1, lon_2)
points.append(point)

In [17]:
random_afghan_point = point

In [18]:
# Create map
map_ = folium.Map(location = afghanistan_center_coordinates,
               zoom_start = 2)
folium.GeoJson(afghanistan_data).add_to(map_)
folium.LayerControl().add_to(map_);

In [19]:
# Preparing for visualization with Timestamped Geo Json
date = datetime.now()
dates = [(date + timedelta(minutes = delta * 5)).strftime("%Y-%m-%dT%H:%M:%S") for delta in range(len(points))]
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates':  [[point.y, point.x]],
        },
        'properties': {
            'times': [date]
        }
    }
    for point, date in zip (points, dates)
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(map_);

In [20]:
map_

## Задание №2. Качество жизни (20 баллов).

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [21]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim
from itertools import chain 
from rtree import index

In [22]:
overpass = Overpass()
nominatim = Nominatim()

#### 1. Расстояние от точки до 5 ближайших банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости

In [23]:
# Get all countries
response = overpass.query('relation[admin_level=2][type=boundary][boundary=administrative];out;', timeout = 120)
# Filter data
countries_data = {}
for country in response.elements():
    if country.id() % 10 == 0:
        countries_data[country.id() + 3600000000] = country.tags()['name:en'] 

In [24]:
residential_buildings_tags = "apartments|bungalow|cabin|detached|dormitory|farm|ger|hotel|house|houseboat|residential|semidetached_house|static_caravan|terrace"

In [25]:
# Count residential buildings in target countries
buildings_count = dict.fromkeys(countries_data.keys())
for id_ in countries_data.keys():
    query = overpassQueryBuilder(area=id_, elementType=['node'], selector=f'"building"~"{residential_buildings_tags}"')
    counter = overpass.query(query, timeout=120).countElements()
    buildings_count[id_] = counter

In [26]:
# Get country with the biggest number of resudential buildings
country_id = max(buildings_count, key = lambda k: buildings_count[k])
print(countries_data[country_id], '- country with the biggest number of resudential buildings')

Malawi - country with the biggest number of resudential buildings


In [27]:
# Get all atms in target country
response = overpass.query(f'area({country_id});(nwr[amenity=atm](area););out ids geom;', timeout=120)

In [28]:
# Find 5 closest atms to afghan random point
idx = index.Index()
for point in response.elements():
    idx.insert(point.id(), point.geometry().coordinates)
nearest_atms = idx.nearest((random_afghan_point.x, random_afghan_point.y), 5, objects = True)

In [29]:
# Summ distances
result = sum([haversine((p.bbox[0], p.bbox[1]), (random_afghan_point.x, random_afghan_point.y), unit = Unit.METERS) for p in nearest_atms])

In [30]:
print('Result sum', result)

Result sum 34920891.24153749


#### 2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице

In [31]:
max_pharmacy = 0
for id_ in countries_data:
    # Get ciuntries capital
    query = f'area({id_});(nwr[capital=yes](area); nwr[capital=country](area); nwr[capital=2](area); nwr[admin_level=2](area);); out body;'
    res = overpass.query(query)
    if (len(res.elements()) == 0):
        print(f'Unable to find capital city for {countries_data[id_]}')
        continue
    capital = res.elements()[0].tags()['name:en']
    # For each capital city get it's area id
    areaId = nominatim.query(capital).areaId()
    if (areaId == None):
        print(f'Unable to find area id for {capital}')
        continue
    # Get all pharmacies in capital
    query = f'area({id_});(nwr[shop=chemist](area);nwr[amenity=pharmacy](area);); out body;'
    result = overpass.query(query).countElements()
    if (result > max_pharmacy):
        max_pharmacy = result
        country_id = id_ 
print(countries_data[country_id], '- country with the biggest number of pharmacies in capital')

Unable to find area id for Canberra
Unable to find area id for Mbabane
Unable to find capital city for São Tomé and Príncipe
Unable to find capital city for Tokelau
Taiwan - country with the biggest number of pharmacies in capital


In [32]:
# Get all schools in target country
query = overpassQueryBuilder(area=country_id, elementType=['node', 'way'], selector='amenity=school',  includeGeometry=True)
response = overpass.query(query)

In [33]:
# Find 5 closest schools to afghan random point
idx = index.Index()
for el in response.elements():
    if (el.geometry().type == 'Pont'):
        idx.insert(el.id(), el.geometry().coordinates)
    if (el.geometry().type == 'Polygon'):
        polygon_center = Polygon(list(chain.from_iterable(el.geometry().coordinates))).centroid
        idx.insert(el.id(), [polygon_center.x, polygon_center.y])
nearest_schools = idx.nearest((random_afghan_point.x, random_afghan_point.y), 5, objects = True)

In [34]:
# Summ distances
result += sum([haversine((p.bbox[0], p.bbox[1]), (random_afghan_point.x, random_afghan_point.y), unit = Unit.METERS) for p in nearest_schools])

In [35]:
print('Result sum', result)

Result sum 42972593.44071838


#### 3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части 

In [36]:
# Function to get array of points of country bounds
def get_county_bounds(county_name):
    resp = overpass.query(f'relation[admin_level=2][type=boundary][boundary=administrative]["name:en"="{county_name}"];out geom;')
    if resp.elements()[0].geometry().type == 'Polygon':
        bound_points = np.array(resp.elements()[0].geometry().coordinates[0])
        bound_points.reshape(bound_points.shape[-2], 2)
    elif resp.elements()[0].geometry().type == 'MultiPolygon':
        bound_points = np.array([point for polygon in resp.elements()[0].geometry().coordinates for point in polygon[0]])
    return bound_points

In [37]:
# Function to build triangle using extreme points ща сщгтекн ищгтвы
def get_south_part_polygon(county_name):
    bound_points = get_county_bounds(county_name)
    max_lon = max(bound_points, key=lambda point:point[0])
    max_lat = max(bound_points, key=lambda point:point[1])
    min_lon = min(bound_points, key=lambda point:point[0])
    min_lat = min(bound_points, key=lambda point:point[1])
    return Polygon([[max_lon[0], max_lat[1]], [max_lon[0], min_lat[1]], [min_lon[0], min_lat[1]]])

In [38]:
# Function to count all points within polygon
def count_points_within_polygon(response, polygon):
    counter = 0
    for el in response.elements():
        if el.geometry().type == 'Point':
            point = Point(el.geometry().coordinates[0], el.geometry().coordinates[1])
        elif el.geometry().type == 'Polygon':
            centroid = Polygon(el.geometry().coordinates[0]).centroid
            point = Point(centroid.x, centroid.y)
        if (point.within(polygon)):
            counter += 1
    return counter

In [40]:
max_value = -1
for id_ in countries_data:
    south_part = get_south_part_polygon(countries_data[id_])
    # Get all bus stations
    bus_query = overpassQueryBuilder(area=id_, elementType=['node', 'way'], selector='amenity=bus_station', out='geom')
    # Get all train stations
    train_query = overpassQueryBuilder(area=id_, elementType=['node', 'way'], selector='building=train_station', out='geom')
    bus_stations_amount = count_points_within_polygon(overpass.query(bus_query), south_part)
    train_stations_amount = count_points_within_polygon(overpass.query(train_query), south_part)
    current_value = train_stations_amount / (bus_stations_amount + 0.001)
    if (current_value > max_value):
        max_value = current_value
        country_id = id_
print(countries_data[country_id], '- is the result country')

Australia - is the result country


In [41]:
# Get all cinemas in target country
query = overpassQueryBuilder(area=country_id, elementType=['node', 'way'], selector='amenity=cinema',  includeGeometry=True)
response = overpass.query(query)

In [42]:
# Find 5 closest cinemas to afghan random point
idx = index.Index()
for el in response.elements():
    if (el.geometry().type == 'Pont'):
        idx.insert(el.id(), el.geometry().coordinates)
    if (el.geometry().type == 'Polygon'):
        polygon_center = Polygon(list(chain.from_iterable(el.geometry().coordinates))).centroid
        idx.insert(el.id(), [polygon_center.x, polygon_center.y])
nearest_cinemas = idx.nearest((random_afghan_point.x, random_afghan_point.y), 5, objects = True)

In [43]:
# Summ distances
result += sum([haversine((p.bbox[0], p.bbox[1]), (random_afghan_point.x, random_afghan_point.y), unit = Unit.METERS) for p in nearest_cinemas])

In [44]:
print('Result sum', result)

Result sum 78577549.44103174
